# Exercise Sheet \#3


## Exercise 1. Quotes: manual scraping

In this exercise, you are required to compile a dataset of biographies taken from http://quotes.toscrape.com.
Recall this website displays 10 quotes per page, together with a link to their author's biography. This will be a step by step guide.

#### 1.1 Getting URLs of authors' pages

To get a list of URLs pointing at author pages, you will process quotes' pages. 

To do so, first complete the function get_links below which expects as parameter:

* `url` the URL of a page from quotes.toscrape.com

and returns:

* `authors` the list of links to author pages contained in the given quotes' page (beware of duplicates!)

In [1]:
import requests, re
from bs4 import BeautifulSoup

BASE_URL = 'http://quotes.toscrape.com'

def get_links(url):
    authors = []
    # Get page located at url:
    
    #Get all links corresponding to authors:
    
    #Loop over these:
    
        #if a link is not in authors, add it:
        
    #Return results
    return authors

#Test:
authors = get_links(BASE_URL)
print(authors)

[]


#### 1.2 iterate over pages of quotes

In a second step, fill the `collect` function below, which will iteratively collect author links. This function will take as input parameters:
- `url`: the starting url from which to collect links,
- `authors`: the list of links to be updated
- `limit`: the number of pages to visit (default being `None`, which means visit all pages)

In [2]:
def collect(url, authors, limit=None):
    #Add links contained in page located at url to the authors being computed
    authors.extend([x for x in get_links(url) if x not in authors])
    #If no limit is given or limit > 1

        # Get page located at url:

        # Get url of next page

        # recursively collect links (if any)

# Test
authors = []
collect(BASE_URL, authors, limit=2)
print(authors)

[]


#### Question 1.3 : get actual biographies

For each of the links computed in the previous question, retrieve the corresponding webpage and extract the biography it contains. To do so, fill the `get_biography` function below. It will feed a list of dictionaries of the following form:
```python
bios = [{name: '...', birth_date: '...', birth_place: '...', bio: '...'}, ...]
```

In [3]:
def get_biography(url):
    # Get page located at URL and parse it
    
    # Get name with BeautifulSoup
    name = None
    # Get birth date
    birth_date = None
    # Get birth place
    birth_place= None
    # Get bio
    bio = None
    return {'name':name, 'birth_date': birth_date, 'birth_place': birth_place, 'bio': bio}

def get_bios(urls):
    bios = []
    for u in urls:
        bios.append(get_biography(u))
    return bios

#Test
bios=get_bios(authors)
print(bios)

[]


#### Question 1.4: save your dataset

Finally, write a `save` function which takes as an input a list of biographies as computed above and save them in JSON on disk (the filename being an input parameter).

In [4]:
import json

def save(filename, dataset):
    # Open output file
    # write data in JSON format
    pass #remove when ready

save('bios.json', bios)

## Exercise 2. Let's use Scrapy now!

Here the goal is to play with scrapy. Let's look at the wikipedia article https://en.wikipedia.org/wiki/List_of_French_artists. Let's say, we want to extract all names of artists from here with links to their corresponding wikipedia pages and the first paragraph about them.

You will find a file called `Exercise_sheet_3_scrapy.py`. Can you fill in the gaps in this script?


In addition to the Scrapy documentation I highly recommend you to look at possible selectors: https://www.w3schools.com/cssref/css_selectors.php

In [12]:
pip install scrapy

     ---------------------------------------- 0.0/272.9 kB ? eta -:--:--
     - -------------------------------------- 10.2/272.9 kB ? eta -:--:--
     - -------------------------------------- 10.2/272.9 kB ? eta -:--:--
     - -------------------------------------- 10.2/272.9 kB ? eta -:--:--
     ----------- ------------------------- 81.9/272.9 kB 459.5 kB/s eta 0:00:01
     -------------- --------------------- 112.6/272.9 kB 469.7 kB/s eta 0:00:01
     ------------------------- ---------- 194.6/272.9 kB 695.5 kB/s eta 0:00:01
     -----------------------------------  266.2/272.9 kB 820.5 kB/s eta 0:00:01
     ------------------------------------ 272.9/272.9 kB 765.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.1 MB 5.7 MB/s eta 0:00:01
     ------ --------------------------------- 0.5/3.1 MB 4.7 MB/s eta 0:00:01
     --------- ------------------------------ 0.8/3.1 MB 4.5 MB/s eta 0:00:01


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'twisted_iocpsupport.iocpsupport' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for twisted-iocpsupport
ERROR: Could not build wheels for twisted-iocpsupport, which is required to install pyproject.toml-based projects


In [11]:
import scrapy
import json

class WikipediaSpider(scrapy.Spider):
        name = "wikipedia"
        
        start_urls = ["https://en.wikipedia.org/wiki/List_of_French_artists"]
        
        def parse(self, response):
            list_els = response.css('div.mw-parser-output>ul>li>a::attr(href)').getall() #get all links which are in lists on the page
            list_see_also = response.css('div.mw-parser-output>ul:last-of-type>li>a::attr(href)').getall() #get the links in see also (they are not about the disctinct authors)
            res_list = list(set(list_els) - set(list_see_also))
            for link in res_list:
                if link.startswith("/wiki/"): #check that the link actually exists and is not red
                    yield response.follow(link, callback=self.parse_artist)
                    
        def parse_artist(self, response):
            #url = #get url of the page
            name = response.css('span.mw-page-title-main::text').get() # get name of the artist
            paragraph = response.css('div.mw-parser-output>p:nth-of-type(1)').xpath('string(.)').get() # get the first paragraph
            yield {'url': url,
                   'name': name,
                   'paragraph': paragraph}
            
        if __name__=='__main__':
            import scrapy.crawler
            
            process = scrapy.crawler.CrawlerProcess({
                'USER_AGENT': 'Mozilla/5.0',
                'FEEDS': {
                    "artists.json": {"format": "json"},
                },
            
            })
            process.crawl(WikipediaSpider)
            process.start()
            process.stop()


ModuleNotFoundError: No module named 'scrapy'